In [ ]:
!pip install ucimlrepo
!pip install pandas
!pip install scipy
!pip install tensorflow==2.10.0
!pip install keras==2.10.0
!pip install kormos


In [ ]:
from google.colab import files

# This will prompt you to upload a file
uploaded = files.upload()

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import kagglehub
import random
import datetime

In [ ]:

df = pd.read_csv('heart_disease_uci.csv')
df1 = pd.read_csv('bank.csv')
df2 = pd.read_csv('winequality-red.csv')

seed = 1234
np.random.seed(seed)

# Access data
##############################################################################################################################

# Heart Features
# Heart Target

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
df_encoded = pd.get_dummies(df, columns=categorical_columns)
#col_means = {col:  np.mean(df_encoded[col]) for col in df_encoded.columns}
#df_encoded = df_encoded.fillna(col_means)
X_heart = df_encoded.drop(columns=['num'])
y_heart = df_encoded['num']

# Bank Features
categorical_columns = df1.select_dtypes(include=['object', 'category']).columns
df1_encoded = pd.get_dummies(df1, columns=categorical_columns)
#col_means = {col:  np.mean(df_encoded[col]) for col in df1_encoded.columns}
#df1_encoded = df1_encoded.fillna(col_means)
X_bank = df1_encoded.drop(columns=['deposit_no', 'deposit_yes'])
# Bank Target
y_bank = df1_encoded['deposit_yes']

# Ban Features
categorical_columns = df2.select_dtypes(include=['object', 'category']).columns
df2_encoded = pd.get_dummies(df2, columns=categorical_columns)
#col_means = {col:  np.mean(df2_encoded[col]) for col in df2_encoded.columns}
#df2_encoded = df2_encoded.fillna(col_means)
X_wine = df2_encoded.drop(columns=['quality'])
# Bank Target
y_wine = df2_encoded['quality']

##############################################################################################################################

#Print to check
##############################################################################################################################

print("Heart Disease Features:\n", X_heart)

print("Heart Disease Target:\n", y_heart)

print("\n####################################################################################\n")
print("Bank Marketing Features:\n", X_bank)

print("Bank Marketing Target:\n", y_bank)
print("\n####################################################################################\n")
print("Wine Quality:\n", X_wine)

print("Wine Quality Target:\n", y_wine)
print("\n")

##############################################################################################################################

# Data Split
##############################################################################################################################
# Train/Test
X_Htrain, X_Htest, y_Htrain, y_Htest = train_test_split(X_heart, y_heart, test_size=0.2, random_state=seed)
X_Wtrain, X_Wtest, y_Wtrain, y_Wtest = train_test_split(X_wine, y_wine, test_size=0.2, random_state=seed)
X_Btrain, X_Btest, y_Btrain, y_Btest = train_test_split(X_bank, y_bank, test_size=0.2, random_state=seed)

# Train/Val
X_Htrain, X_Hval, y_Htrain, y_Hval = train_test_split(X_Htrain, y_Htrain, test_size=0.3, random_state=seed)
X_Wtrain, X_Wval, y_Wtrain, y_Wval = train_test_split(X_Wtrain, y_Wtrain, test_size=0.3, random_state=seed)
X_Btrain, X_Bval, y_Btrain, y_Bval = train_test_split(X_Btrain, y_Btrain, test_size=0.3, random_state=seed)

###############################################################################################################################

# Print to check
###############################################################################################################################
print("Heart Disease Dataset:")
print("Training features shape:", X_Htrain.shape)
print("Validation features shape:", X_Hval.shape)
print("Testing features shape:", X_Htest.shape)
print("Training target shape:", y_Htrain.shape)
print("Validation target shape:", y_Hval.shape)
print("Testing target shape:", y_Htest.shape)

print("\nWine Quality Dataset:")
print("Training features shape:", X_Wtrain.shape)
print("Validation features shape:", X_Wval.shape)
print("Testing features shape:", X_Wtest.shape)
print("Training target shape:", y_Wtrain.shape)
print("Validation target shape:", y_Wval.shape)
print("Testing target shape:", y_Wtest.shape)

print("\nBank Dataset:")
print("Training features shape:", X_Btrain.shape)
print("Validation features shape:", X_Bval.shape)
print("Testing features shape:", X_Btest.shape)
print("Training target shape:", y_Btrain.shape)
print("Validation target shape:", y_Bval.shape)
print("Testing target shape:", y_Btest.shape)

#################################################################################################################################

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kormos.models import BatchOptimizedSequentialModel

def build_model(optim_type, num_classes, learning_rate_init, batch_size, hidden_layer_sizes, activation):
  model = BatchOptimizedSequentialModel()
  for layer_size in hidden_layer_sizes:
    model.add(keras.layers.Dense(layer_size, activation=activation))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=optim_type, metrics=['accuracy'])
  return model


In [ ]:
def random_hyperparamaters():
  learning_rate = random.choice([0.0001, 0.001, 0.01, 0.1])
  batch_size = random.choice([16, 32, 64, 128])
  num_layers = random.choice([1, 2, 3, 4])
  nodes_per_layer = random.choice([32, 64, 128, 256])
  activation_function = random.choice(['relu', 'tanh'])

  return {'learning rate': learning_rate, 'batch size' : batch_size, 'num of layers' : num_layers, 'nodes per layer': nodes_per_layer, 'activation function' : activation_function }


def generate_sample(n):

    sample = set()

    while(len(sample) < n):

      sample.add(tuple(random_hyperparamaters().items()))

    return [dict(comb) for comb in sample]

def map_hyperparameters(params):

    hidden_layer_sizes = tuple([params['nodes per layer']] * params['num of layers'])
    activation = params['activation function']

    return {
        'learning_rate_init': params['learning rate'],
        'batch_size': params['batch size'],
        'hidden_layer_sizes': hidden_layer_sizes,
        'activation': activation
    }

# Function to train and evaluate the MLP with the best hyperparameters
def train_mlp_with_hyperparameters(best_hyperparams, X_tr, y_tr, X_val, y_val, X_te, y_te, seed):
      mlp = MLPClassifier(**best_hyperparams, shuffle=True, random_state=seed, verbose=False)
      mlp.fit(X_tr, y_tr)
      train_accuracy = mlp.score(X_tr, y_tr)
      val_accuracy = mlp.score(X_val, y_val)
      test_accuracy = mlp.score(X_te, y_te)

      print('Best hyperparameters performance:')
      print(f'Training accuracy: {train_accuracy}')
      print(f'Validation accuracy: {val_accuracy}')
      print(f'Test accuracy: {test_accuracy}')
      return train_accuracy, val_accuracy, test_accuracy

def run_test(X_tr, y_tr, X_val, y_val, X_te, y_te):

  sample = generate_sample(50)


  bfgs_results = []
  cg_results = []

  imputer = SimpleImputer(strategy='mean')
  X_tr = imputer.fit_transform(X_tr)
  X_val = imputer.transform(X_val)
  X_te = imputer.transform(X_te)

  scaler = StandardScaler()
  X_tr = scaler.fit_transform(X_tr)
  X_val = scaler.transform(X_val)
  X_te = scaler.transform(X_te)

  bfgs_tot = None
  cg_tot = None

  for params in sample:
      mapped_params = map_hyperparameters(params)
      start = datetime.datetime.now()
      bfgs = build_model('L-BFGS-B', len(np.unique(y_tr)), **mapped_params)
      bfgs.build(X_tr.shape)
      bfgs_history = bfgs.fit_batch(X_tr, y_tr, validation_data=(X_val, y_val), epochs=10, steps_per_epoch=50, batch_size = mapped_params['batch_size'])
      train_accuracy = np.min(bfgs.history.history['loss'])
      val_accuracy = np.max(bfgs.history.history['val_accuracy'])
      bfgs_results.append((params, train_accuracy, val_accuracy))
      if bfgs_tot is None:
        bfgs_tot = (datetime.datetime.now() - start).total_seconds()
      else:
        bfgs_tot += (datetime.datetime.now() - start).total_seconds()
      start = datetime.datetime.now()
      cg = build_model('CG', len(np.unique(y_tr)), **mapped_params)
      cg.build(X_tr.shape)
      cg_history = cg.fit_batch(X_tr, y_tr, validation_data=(X_val, y_val), epochs=10, steps_per_epoch=50, batch_size = mapped_params['batch_size'])
      train_accuracy = np.min(cg_history.history['loss'])
      val_accuracy = np.max(cg_history.history['val_accuracy'])
      cg_results.append((params, train_accuracy, val_accuracy))
      if cg_tot is None:
        cg_tot = (start - datetime.datetime.now()).total_seconds()
      else:
        cg_tot += (start - datetime.datetime.now()).total_seconds()

  print("BFGS Time: ", bfgs_tot/len(bfgs_results))
  print("CG Time: ", cg_tot/len(cg_results))

  # Select the best hyperparameters based on validation accuracy
  best_params_bfgs, best_train_accuracy_bfgs, best_val_accuracy_bfgs = max(bfgs_results, key=lambda x: x[2])
  best_hyperparams_bfgs = map_hyperparameters(best_params_bfgs)

  print("Best BFGS hyperparameters:", best_params_bfgs)

  best_params_cg, best_train_accuracy_cg, best_val_accuracy_cg = max(cg_results, key=lambda x: x[2])
  best_hyperparams_cg = map_hyperparameters(best_params_cg)

  print("Best CG hyperparameters:", best_params_cg)

  # Check your results
  print('BFGS Results: ')
  train_mlp_with_hyperparameters(best_hyperparams_bfgs, X_tr, y_tr, X_val, y_val, X_te, y_te, seed)

  print('CG Results: ')
  train_mlp_with_hyperparameters(best_hyperparams_cg, X_tr, y_tr, X_val, y_val, X_te, y_te, seed)

print('Running for Heart Disease:')
run_test(X_Htrain, y_Htrain, X_Hval, y_Hval, X_Htest, y_Htest)

In [ ]:
print('Running for Wine Quality:')
run_test(X_Wtrain, y_Wtrain, X_Wval, y_Wval, X_Wtest, y_Wtest)

In [ ]:
print('Running for Bank:')
run_test(X_Btrain, y_Btrain, X_Bval, y_Bval, X_Btest, y_Btest)